<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2019 al 2023</font>
</p>

# Tabla de contenidos

1. [*QTimer*](#QTimer)
    1. [Ejemplo-multiples-QTimer](#Ejemplo-multiples-QTimer)
    2. [`singleShot`](#singleShot)
2. [Ejemplo Aplicado](#Ejemplo-Aplicado)

## QTimer

Otra herramienta que existe para generar comportamiento de concurrencia dentro de PyQt es la clase `QTimer`. Los objetos de esta clase, a diferencia de los `QThread`, no son un símil a `Timer` del módulo `threading`: `Timer` después de un tiempo específico ejecuta una subrutina una única vez, mientras que `QTimer` ejecuta una subrutina cada cierto tiempo determinado periódicamente, repitiendo la subrutina una y otra vez.

Este tipo de comportamiento es simulable utilizando `QThread`, al definir código para un *thread* de la forma:

```python
def run(self):
    while True:
        # Lo que quiero que el QThread haga en cada iteración
        time.sleep(self.tiempo)
```

... y debería obtener el mismo resultado descrito al usar `QTimer`, lo que es cierto. La desventaja de realizar esto en comparación con usar un `QTimer` es que estos últimos están construidos para efectuar este comportamiento, mientras que los `QThreads`, como los *threads* en general, están construidos para que acaben eventualmente. Luego, el implementar un *thread* usando el código anterior nos obliga a usar el método `terminate`, que se considera **mala práctica** al forzar un *thread* a terminar, en vez de que este termine por su cuenta.

Por su lado, `QTimer` provee métodos para comenzar (`start`) y detener (`stop`) la ejecución periódica de buena manera. Tras inicializar un `QTimer`, se le asigna mediante `setInterval` el tiempo en milisegundos que durará el periodo entre ejecuciones, y mediante el atributo (y señal) `timeout` se puede conectar a la subrutina que se efectuará una y otra vez: `timer.timeout.connect(subrutina)`.


En el siguiente ejemplo crearemos un reloj digital que actualizará el tiempo cada 1 segundo. Este código se encuentra en el archivo `2-pyqt-qtimer-ejemplo_1.py`

```python
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
from PyQt6.QtCore import QTimer
from PyQt6.QtGui import QFont
import sys
import datetime


class RelojDigital(QWidget):
    def __init__(self):
        super().__init__()

        # Crear label encargado de mostrar la hora
        self.label_timer = QLabel()
        self.label_timer.setFont(QFont("Times", 50))

        # Crear layout vertical para nuestro label
        layout = QVBoxLayout()
        layout.addWidget(self.label_timer)
        self.setLayout(layout)

        # Crear nuestro QTimer encargado de actualizar el tiempo cada 1 segundo
        timer = QTimer(self)
        timer.timeout.connect(self.mostrar_hora)
        timer.setInterval(1000)
        timer.start()

        # Definir título y tamaño ventana
        self.setWindowTitle("Reloj Digital con QTimer")
        self.setGeometry(100, 100, 250, 100)

        # Ejecutar el método para mostrar hora por primera vez
        self.mostrar_hora()

        # Mostrar ventana
        self.show()

    def mostrar_hora(self):
        # Obtener hora actual
        hora_actual = datetime.datetime.now().time()
        # Actualizar texto del label
        self.label_timer.setText(hora_actual.strftime("%H:%M:%S %p"))
```

### Ejemplo multiples QTimer 

A continuación se muestra el último ejemplo de `QThread` visto en el contenido anterior, pero adaptado a `QTimer` en vez de `QThread`. Este código se encuentra en el archivo `2-pyqt-qtimer-ejemplo_2.py`

```python
import sys
from PyQt6.QtCore import pyqtSignal, QObject, QTimer
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiTimer(QObject):
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, indice, tiempo):
        super().__init__()
        self.indice = indice
        self.tiempo = tiempo
        self.indice_actual = 0
        self.timer = QTimer(self)

        # Acá se asigna el tiempo de duración del periodo entre ejecuciones
        self.timer.setInterval(int(tiempo * 1000))
        # Acá se conecta la subrutina que se ejecutará
        self.timer.timeout.connect(self.enviar_dato)

    def enviar_dato(self):
        if self.indice_actual <= 9:
            self.senal_actualizar.emit(self.indice, str(self.indice_actual))
            self.indice_actual += 1
        else:
            self.senal_actualizar.emit(self.indice, "Status: Qtimer terminado")
            self.timer.stop()

    def comenzar(self):
        self.timer.start()

    def sigue_andando(self):
        return self.timer.isActive()


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.timers = []
        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un Qtimer cada uno
        self.labels = {i: QLabel("Status: esperando Qtimer", self) for i in range(1, 6)}
        self.boton = QPushButton("Ejecutar Qtimers", self)
        self.boton.clicked.connect(self.ejecutar_timers)

        for i in range(1, 6):
            self.labels[i].setGeometry(10, (i - 1) * 30, 230, 30)

        self.boton.setGeometry(10, 150, 230, 30)
        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qtimers")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_timers(self):
        """
        Este método crea cinco timers cada vez que se presiona el botón en la
        interfaz. Los timers recibirán como argumento el índice del label
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        if any([timer.sigue_andando() for timer in self.timers]):
            return

        self.timers = []
        for i in range(1, 6):
            timer = MiTimer(i, i / 10)
            # Se conecta la señal emitida por el timer a un método
            # de la ventana
            timer.senal_actualizar.connect(self.actualizar_labels)
            self.timers.append(timer)
            timer.comenzar()

    def actualizar_labels(self, indice, texto):
        """
        Este método actualiza el label correspondiente según los datos
        enviados desde un timer através del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)

```

### `singleShot`

Por defecto, un `QTimer` se ejecutará periódicamente cada vez que se cumpla el tiempo indicado en `setInterval`. No obstante, puede ocurrir una situación donde queremos que el `QTimer` ejecute su función una **única vez** después de transcurrir el tiempo esperado. 

Un ejemplo podría ser dónde queremos que la ventana se cierre después de 10 segundos. Para modelar este ejemplo con `QTimer`, podríamos hacer:
- Un `QTimer` que se ejecuta cada 1 segundos y posee un contador. Cuando este contador llega a 10, se cierra la ventana.
- Un `QTimer` que se ejecuta cada 10 segundos. Una vez ejecutada su función, el mismo hace `stop` para no volver a ejecutarse. Además, aprovecha de cerrar la ventana.


No obstante, también existe otra forma más que es utilizar `singleShot`. Este método nos permite indicar a `pyqt` que el `QTimer` solo se ejecutará 1 vez y luego debe ser detenido. De este modo no necesitamos incluir un `stop()`.

A continuación vamos a tomar el ejemplo del `RelojAnalogico` pero vamos a incluir 2 `QTimer` que son `singleShot`. El primero, va a esperar 5 segundos y hará `hide` de la ventana, mientras que el segundo esperará 8 segundos y hará `show` de la ventana.

Este código se encuentra en el archivo `2-pyqt-qtimer-ejemplo_3.py`

```python
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
from PyQt6.QtCore import QTimer
from PyQt6.QtGui import QFont
import sys
import datetime


class RelojDigital(QWidget):
    def __init__(self):
        super().__init__()

        # Crear label encargado de mostrar la hora
        self.label_timer = QLabel()
        self.label_timer.setFont(QFont("Times", 50))

        # Crear layout vertical para nuestro label
        layout = QVBoxLayout()
        layout.addWidget(self.label_timer)
        self.setLayout(layout)

        # Crear nuestro QTimer encargado de actualizar el tiempo cada 1 segundo
        timer = QTimer(self)
        timer.timeout.connect(self.mostrar_hora)
        timer.setInterval(1000)
        timer.start()

        # Definir título y tamaño ventana
        self.setWindowTitle("Reloj Digital con QTimer")
        self.setGeometry(100, 100, 250, 100)

        # Ejecutar el método para mostrar hora por primera vez
        self.mostrar_hora()

        # Mostrar ventana
        self.show()
        
        # Creamos un QTimer que despues de 5 segundos va a esconder la ventana
        self.timer_singleshot_hide = QTimer(self)
        self.timer_singleshot_hide.setSingleShot(True)
        self.timer_singleshot_hide.timeout.connect(self.hide)
        self.timer_singleshot_hide.setInterval(5000)
        self.timer_singleshot_hide.start()

        # Creamos otro QTimer que despues de 8 segundos va a mostrar la ventana
        self.timer_singleshot_show = QTimer(self)
        self.timer_singleshot_show.setSingleShot(True)
        self.timer_singleshot_show.timeout.connect(self.show)
        self.timer_singleshot_show.setInterval(8000)
        self.timer_singleshot_show.start()

    def mostrar_hora(self):
        # Obtener hora actual
        hora_actual = datetime.datetime.now().time()
        # Actualizar texto del label
        self.label_timer.setText(hora_actual.strftime("%H:%M:%S %p"))
```

## Ejemplo Aplicado

Ahora, vamos a realizar el mismo ejemplo del contenido anterior pero aplicando `QTimer`.

![](img/ejemplo_aplicado.gif)

En este ejemplo, tendremos diferentes cuadrados que van de un extremo a otro a diferente velocidad. Además, se puede hacer click en un cuadrado para cambiar su color y detener su movimiento. (Mini desafío: intenta detener todos los cuadrados).

Para esto, dentro del directorio `scripts` hay un directorio llamado `ejemplo_aplicado`. En este tenemos los siguientes archivos:

* `main.py`: es el archivo principal a ejecutar.
* `parametros_general.py`: son constantes que se ocuparán tanto en _back-end_ y _front-end_.

**backend**
* `backend/logica_qthread.py`: es el archivo principal del _back-end_ donde se aplica toda la lógica del movimiento de los cuadrados. En este archivo se ocupan `QThread` para modelar cada cuadrado. Este archivo se recomienda ignorar para este ejemplo. Se utilizó en el contenido número 1.
* `backend/logica_qtimer.py`: es el mismo archivo que `logica_qthread.py` pero se utilizan `QTimer` en vez de `QThread`. **Este archivo se utilizará en este ejemplo**.
* `backend/parametros_backend.py`: son constantes que se ocuparán únicamente en _back-end_.

**frontend**
* `frontend/ventana.py`: es el archivo principal del _front-end_ donde se crea la ventana y todo elemento visual.
* `frontend/parametros_backend.py`: son constantes que se ocuparán únicamente en _front-end_.

Para la correcta ejecución de este archivo, debes abrir tu terminal (usar WSL en caso de Windows) y desplazarte con `cd` hasta el directorio `scripts/ejemplo_aplicado/` y luego escribir `python3 main.py`.

**Importante** para utilizar `backend/logica_qtimer.py`, debes:
1. Ir a `main.py`
2. Comentar la línea 7: `from backend.logica_qthread import Juego`
3. Descomentar la línea 8: `from backend.logica_qtimer import Juego`

Para esto vamos a utilizar el _back-end_ que utiliza `QTimer`. Podrás notar que el _front-end_ es el mismo, puesto que una correcta modularización nos permite reutilizar el mismo _front-end_, modificar el _back-end_ y que todo funcione correctamente.